# 토큰화

토큰화(tokenization)는 자연어를 **모델이 이해할 수 있는 또는 모델이 다룰 수있는 기본 단위(Token)** 분할하는 과정을 말한다.   
토큰으로 나누는 단위는 설계에 따라 문장, 어절, 형태소, 서브워드, 문자, 자모/알파벳 등 다양한 방식으로 나눌 수 있다.   
- 예
```bash
원문: "자연어 처리는 재미있다"
토큰화: ["자연어", "처리", "는", "재미있다"]
```

## 토큰화 방식
- **단어 기반 토큰화(Word-Level Tokenization)**
    - 어절(공백으로 구분) 또는 형태소 단위로 단어를 나누는 전통적인 방식이다.
    - **한국어**는 교착어로 하나의 단어에 다양한 조사/어미가 결합된다. 그래서 어절단위로 토큰화할 경우 어휘사전의 크기가 기하급수적으로 늘어나는 문제가 있다.
      - 예) "학교", "학교가", "학교를", "학교에", "학교에서", "학교로", "학교의", ...
    - 이로 인해 미등록어휘(OOV - Out of Vocabulary)의 증가, 같은 의미를 가지는 단어들이 Vocab에 중복 등록, 메모리 낭비, 학습효율성 저하 등 다양한 문제가 생긴다.
    - 그래서 **한국어의 경우 형태소 단위 토큰화**가 필요하다.

- **서브워드 기반(Subword-level) — BPE, WordPiece, Unigram**
    - Transformer 기반 모델(BERT, GPT, LLaMA 등)에서 표준으로 사용하는 방식.
    - 단어를 기준으로 토큰화하지 않고 **문자(character)와 단어(word)의 중간 수준인 서브워드(subword) 단위로 토큰화**한다.
    - **동작 원리**:
        - 자주 등장하는 문자열 조합(서브워드)을 하나의 토큰으로 구성한다.
        - 빈도가 높은 단어는 하나의 토큰으로, 빈도가 낮거나 희귀한 단어는 여러 서브워드로 분할한다.
    - **예시**:
        ```bash
        입력: "나는 밥을 먹었습니다. 나는 어제 밥을 했습니다."
        
        서브워드 토큰화 결과 (예시):
        ["나는", "밥", "을", "먹", "었", "습니다", ".", "나는", "어제", "밥", "을", "하", "었", "습니다", "."]
        ```
    - **장점**:
        - **미등록 단어(OOV) 문제 해결**: 모든 단어를 서브워드 조합으로 표현 가능
        - **어휘 사전 크기 최적화**: 단어 단위보다 작고, 문자 단위보다 효율적
        - **다국어 지원**: 언어에 구애받지 않는 범용적 토큰화
        - **형태론적 의미 포착**: 접두사, 접미사 등의 의미를 학습 가능
    - **주요 알고리즘**:
        - **BPE (Byte Pair Encoding)**: 가장 빈번한 연속 바이트/문자 쌍을 반복적으로 병합
        - **WordPiece**: BERT에서 사용, BPE와 유사하지만 likelihood 기반으로 병합
        - **Unigram**: 확률 모델 기반으로 최적의 서브워드 분할 선택


# 한국어 형태소 분석기

- kiwipiepy와 konlpy 는 대표적인 한국어 형태소 분석기이다.

## kiwipiepy
**kiwipiepy**는 C++로 구현된 한국어 형태소 분석기 Kiwi(Korean Intelligent Word Identifier)를 Python 환경에서 사용할 수 있도록 한 라이브러리이다. 

- 빠른 속도  
- 최신 품사 체계 지원  
- 사용자 사전 확장 용이  
- 최근 가장 널리 쓰이는 한국어 토크나이저 중 하나이다.
- https://github.com/bab2min/kiwipiepy
  
### 설치 방법

```bash
pip install kiwipiepy
```


In [2]:
!uv pip install kiwipiepy

Using Python 3.12.12 environment at: c:\Users\Playdata\Documents\SKN21\.venv
Resolved 5 packages in 10ms
Installed 4 packages in 151ms
 + kiwipiepy==0.22.1
 + kiwipiepy-model==0.22.0
 + numpy==2.3.5
 + tqdm==4.67.1


### 주요 클래스 및 함수

#### Kiwi 클래스
- Kiwi의 핵심 클래스이며, 형태소 분석과 토큰화 기능을 모두 제공한다.
- Kiwi 품사는 세종 말뭉치를 기반으로 한다.
  - 품사 시작 글자
  - 체언(명사, 대명사): `N`, 용언(동사, 형용사): `V`, 수식언(관형사, 부사): `M`,  관계언(조사):`J`, 어미: `E`, 기호: `S`
    - https://github.com/bab2min/kiwipiepy?tab=readme-ov-file#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8
- 메소드
  - `tokenzie(text)`: 형태소 분석 기반 토큰화 수행
  - `analyze(text)`: tokenize보다 좀 더 상세한 분석을 진행한다. 여러 분석결과를 조회할 수있다.
  - `add_user_word(word, pos, score)`: 사전에 직접 단어 등록
  - `space(text)`: 띄어 쓰기 교정

In [4]:
import kiwipiepy

In [5]:
from kiwipiepy import Kiwi
from pprint import pprint # 자료구조 출력을 보기좋게 print해줌.

In [ ]:
kiwi = Kiwi()

In [10]:
#############
# 토큰화
#############
text = "나는 자연어 처리를 공부한다.\n내일은 뭘 공부할까?"
tokens = kiwi.tokenize(text)
pprint(tokens)

[Token(form='나', tag='NP', start=0, len=1),
 Token(form='는', tag='JX', start=1, len=1),
 Token(form='자연어 처리', tag='NNP', start=3, len=6),
 Token(form='를', tag='JKO', start=9, len=1),
 Token(form='공부', tag='NNG', start=11, len=2),
 Token(form='하', tag='XSV', start=13, len=1),
 Token(form='ᆫ다', tag='EF', start=13, len=2),
 Token(form='.', tag='SF', start=15, len=1),
 Token(form='내일', tag='NNG', start=17, len=2),
 Token(form='은', tag='JX', start=19, len=1),
 Token(form='뭐', tag='NP', start=21, len=1),
 Token(form='ᆯ', tag='JKO', start=21, len=1),
 Token(form='공부', tag='NNG', start=23, len=2),
 Token(form='하', tag='XSV', start=25, len=1),
 Token(form='ᆯ까', tag='EF', start=25, len=2),
 Token(form='?', tag='SF', start=27, len=1)]


In [ ]:
# Token 객체에서 속성값들 조회
for token in tokens:
    r = f"토큰문자열: {token.form}, 원형(lemma): {token.lemma}, 품사(Tag): {token.tag}\
 시작위치 : {token.start}, 글자수: {token.len}, 토큰이 있는 행번호:{token.line_number}\
 몇번째 문장에 있는지: {token.sent_position}, 문장에서 몇번째 어절인지: {token.word_position}"
    print(r)
    # print("\n")

토큰문자열: 나, 원형(lemma): 나, 품사(Tag): NP 시작위치 : 0, 글자수: 1, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 0
토큰문자열: 는, 원형(lemma): 는, 품사(Tag): JX 시작위치 : 1, 글자수: 1, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 0
토큰문자열: 자연어 처리, 원형(lemma): 자연어 처리, 품사(Tag): NNP 시작위치 : 3, 글자수: 6, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 1
토큰문자열: 를, 원형(lemma): 를, 품사(Tag): JKO 시작위치 : 9, 글자수: 1, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 2
토큰문자열: 공부, 원형(lemma): 공부, 품사(Tag): NNG 시작위치 : 11, 글자수: 2, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 3
토큰문자열: 하, 원형(lemma): 하, 품사(Tag): XSV 시작위치 : 13, 글자수: 1, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 3
토큰문자열: ᆫ다, 원형(lemma): ᆫ다, 품사(Tag): EF 시작위치 : 13, 글자수: 2, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 3
토큰문자열: ., 원형(lemma): ., 품사(Tag): SF 시작위치 : 15, 글자수: 1, 토큰이 있는 행번호:0 몇번째 문장에 있는지: 0, 문장에서 몇번째 어절인지: 3
토큰문자열: 내일, 원형(lemma): 내일, 품사(Tag): NNG 시작위치 : 17, 글자수: 2, 토큰이 있는 행번호:1 몇번째 문장에 있는지: 1, 문장에서 몇번째 어절인지: 0
토큰문자열: 은, 원형(lemma): 은, 품사(Tag): JX 시작위치 : 19, 글자수: 1, 토큰이 있는 행번호:1 몇번째 문장

In [27]:
# 여러 문서를 토큰화할 때는 list로 묶어서 전달.
# 결과: Iterable -> 한번에 한 문서의 결과를 반환.
text_list = ["나는 자연어 처리를 공부한다.", "자연어처리는 NLP라고 한다.", "내일은 뭘 공부할까?"]
tokens = kiwi.tokenize(text_list)
tokens

In [28]:
for token_list in tokens:
    print(token_list)

[Token(form='나', tag='NP', start=0, len=1), Token(form='는', tag='JX', start=1, len=1), Token(form='자연어 처리', tag='NNP', start=3, len=6), Token(form='를', tag='JKO', start=9, len=1), Token(form='공부', tag='NNG', start=11, len=2), Token(form='하', tag='XSV', start=13, len=1), Token(form='ᆫ다', tag='EF', start=13, len=2), Token(form='.', tag='SF', start=15, len=1)]
[Token(form='자연어 처리', tag='NNP', start=0, len=5), Token(form='는', tag='JX', start=5, len=1), Token(form='NLP', tag='SL', start=7, len=3), Token(form='이', tag='VCP', start=10, len=0), Token(form='라고', tag='EC', start=10, len=2), Token(form='하', tag='VV', start=13, len=1), Token(form='ᆫ다', tag='EF', start=13, len=2), Token(form='.', tag='SF', start=15, len=1)]
[Token(form='내일', tag='NNG', start=0, len=2), Token(form='은', tag='JX', start=2, len=1), Token(form='뭐', tag='NP', start=4, len=1), Token(form='ᆯ', tag='JKO', start=4, len=1), Token(form='공부', tag='NNG', start=6, len=2), Token(form='하', tag='XSV', start=8, len=1), Token(form='ᆯ까

In [29]:
###################################
# 상세한 토큰화 분석 - analyze()
###################################
text = "나는 자연어 처리를 공부한다."
result = kiwi.analyze(text, top_n=2)
pprint(result)

[([Token(form='나', tag='NP', start=0, len=1),
   Token(form='는', tag='JX', start=1, len=1),
   Token(form='자연어 처리', tag='NNP', start=3, len=6),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='공부', tag='NNG', start=11, len=2),
   Token(form='하', tag='XSV', start=13, len=1),
   Token(form='ᆫ다', tag='EF', start=13, len=2),
   Token(form='.', tag='SF', start=15, len=1)],
  -32.72097396850586),
 ([Token(form='나', tag='NP', start=0, len=1),
   Token(form='는', tag='JX', start=1, len=1),
   Token(form='자연어', tag='NNP', start=3, len=3),
   Token(form='처리', tag='NNG', start=7, len=2),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='공부', tag='NNG', start=11, len=2),
   Token(form='하', tag='XSV', start=13, len=1),
   Token(form='ᆫ다', tag='EF', start=13, len=2),
   Token(form='.', tag='SF', start=15, len=1)],
  -39.91743469238281)]


In [32]:
text_list = ["나는 자연어 처리를 공부한다.", "자연어처리는 NLP라고 한다.", "내일은 뭘 공부할까?"]
result = kiwi.analyze(text_list, top_n=2)
result
for r in result:
    pprint(r)

[([Token(form='나', tag='NP', start=0, len=1),
   Token(form='는', tag='JX', start=1, len=1),
   Token(form='자연어 처리', tag='NNP', start=3, len=6),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='공부', tag='NNG', start=11, len=2),
   Token(form='하', tag='XSV', start=13, len=1),
   Token(form='ᆫ다', tag='EF', start=13, len=2),
   Token(form='.', tag='SF', start=15, len=1)],
  -32.72097396850586),
 ([Token(form='나', tag='NP', start=0, len=1),
   Token(form='는', tag='JX', start=1, len=1),
   Token(form='자연어', tag='NNP', start=3, len=3),
   Token(form='처리', tag='NNG', start=7, len=2),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='공부', tag='NNG', start=11, len=2),
   Token(form='하', tag='XSV', start=13, len=1),
   Token(form='ᆫ다', tag='EF', start=13, len=2),
   Token(form='.', tag='SF', start=15, len=1)],
  -39.91743469238281)]
[([Token(form='자연어 처리', tag='NNP', start=0, len=5),
   Token(form='는', tag='JX', start=5, len=1),
   Token(form='NLP', tag='SL', start=7, len

In [34]:
##########################
# 띄어쓰기 교정-space()
##########################
text = "자연어처리는재미있는분야이다.또재미있는것은뭐가있을까?"
result = kiwi.space(text)
result

'자연어 처리는 재미있는 분야이다. 또 재미있는 것은 뭐가 있을까?'

In [35]:
text_list = ["자연어처리는재미있는분야이다.또재미있는것은뭐가있을까?", "아버지가방에들어가신다"]
result = kiwi.space(text_list)
for a in result:
    print(a)

자연어 처리는 재미있는 분야이다. 또 재미있는 것은 뭐가 있을까?
아버지가 방에 들어가신다


In [37]:
###################
#문장분리 - split_into_sents()
###################
txt = "NAVER스포츠뉴스엔터사용자 링크내 프로필 이미지 ksryo..내정보 보기네이버톡알림1메일99+메뉴MY팀을 설정해 주세요MY팀 설정본문 바로가기한겨레데뷔는 지명순이 아니죠…프로농구 루키들, 코트 맹활약 예고입력2025.11.27. 오후 4:14수정2025.11.27. 오후 4:26기사원문공감8텍스트 음성 변환 서비스글자 크기 변경공유하기드래프트 이후 닷새 만에강성욱·이유진 등 프로 데뷔흐름 바꾸는 등 적극성 눈길드래프트 뒤 닷새만에 프로에 데뷔한 강성욱(수원 KT). 한국농구연맹 제공“대한민국 가드계에 한 획을 긋겠다”는 야심 찬 계획에 첫 단추를 잘 채웠다.지난 14일 신인드래프트 1라운드 8순위로 수원 케이티(KT) 유니폼을 입은 강성욱(21)이 프로 데뷔전에서 눈도장을 제대로 찍었다강성욱은 드래프트 닷새 만인 19일 2025~2026 남자프로농구(KBL) 안양 정관장과 경기에서 8분57초 동안 5득점 2튄공잡기 3도움주기로 팀 승리에 힘을 보탰다. 후반에 득점과 도움주기를 연달아 기록하며 흐름을 바꾸는 게임 체인저 역할을 해냈다데뷔전에서 수훈선수에도 뽑혔던 강성욱은 “나보다 더 높은 순위로 지명된 선수들을 보며 연습에 더 매진하게 됐다”고 말했다.지난 드래프트에서 지명된 신인들이 하나둘 프로 무대에 뛰어들면서 올 시즌 농구 보는 재미를 더하고 있다. 지명 순위와 관계없는 활약에 이번에는 또 어떤 선수가 프로에서 자신의 가치를 증명할지 매 경기 기대감이 커진다.2라운드 4순위로 울산 현대모비스에 입단한 박정환(22)도 19일 1라운드에서 뽑힌 같은 팀 신인보다 먼저 프로 데뷔전을 치렀다. 서울 에스케이(SK)를 상대로 7분20초를 뛰며 2득점 3도움주기 1가로채기 했다. 상대 이민서의 공을 가로챈 뒤 이를 득점으로 연결하는 등 적극적으로 뛰었다지난 20일 경기에서 프로 데뷔한 원주 디비(DB) 이유진. 한국농구연맹 제공전체 1순위 문유현(가운데)은 25일 대표팀과 연습 경기에서 비공식 데뷔전을 치렀다. 한국농구연맹 제공1라운드 2순위 이유진(20·원주 DB)과 1라운드 5순위 이규태(23·서울 삼성)는 20일 데뷔전에서 루키 맞대결도 펼쳤다이유진은 18분22초를 뛰며 4득점 2튄공잡기 1도움주기, 이규태는 19분 동안 3득점 1튄공잡기 1블록슛을 기록했다. 이규태는 3점으로 프로 첫 득점을 신고하면서 양궁농구를 펼치는 팀에 잘 어우러졌다. 이유진과 이규태가 골 밑에서 튄공잡기 싸움을 하는 등 루키들의 승부가 쫄깃한 재미를 선사했다1라운드 4순위 고양 소노 강지훈(22)도 20일 부산 케이씨씨(KCC)와 경기에서 3분19초 동안 코트를 뛰며 튄공잡기 1개를 기록했다.전체 1순위 문유현(21·안양 정관장)은 지난 25일 농구 대표팀과 연습 경기에서 비공식 데뷔전을 치렀다. 그는 대표팀 에이스 이현중(나가사키)의 수비를 뚫고 3점포를 터뜨리는 등 농구 잘하는 선수가 모인 대표팀을 상대로 6득점 했다. 유도훈 정관장 감독은 “문유현은 빠르면 12월부터 1군에서 모습을 보일 것”이라고 했다.프로농구는 국제농구연맹 월드컵 예선으로 다음 달 3일까지 쉰다. 신인들은 처음으로 10개 구단이 모두 참가한 디(D)리그에 출전해 프로의 세계를 경험 중이다. 강성욱(2경기 평균 16득점), 이규태(2경기 평균 14.5득점) 등은 D리그에서도 훨훨 난다. 휴식기 이후 또 어떤 신인이 자신의 가치를 증명할까.본문의 검색 링크는 AI 자동 인식으로 제공됩니다. 일부에 대해서는 미제공될 수 있고 동일한 명칭이 다수 존재하는 경우에는 전체 검색 결과로 연결될 수 있습니다.오분류 제보하기남지은 기자구독 1,639응원 920구독‘신체접촉’ 피해 마라톤 우승자 “감독이 강하게 잡아 통증 극심했다”태권도가 ‘닌자 훈련’? 캐나다 스포츠채널 부적절 표기 논란한겨레언론사홈 바로가기ⓒ 한겨레신문사 All Rights Reserved. 무단 전재, 재배포, AI 학습 및 활용 금지기사 섹션 분류 가이드[한겨레 후원하기] 시민과 함께 민주주의를!내란 종식 그날까지, 다시 빛의 혁명 ▶참여하기주요뉴스해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다."
result = kiwi.split_into_sents(txt)
result

[Sentence(text='NAVER스포츠뉴스엔터사용자 링크내 프로필 이미지 ksryo..', start=0, end=35, tokens=None, subs=[]),
 Sentence(text='내정보 보기네이버톡알림1메일99+메뉴MY팀을 설정해 주세요', start=35, end=67, tokens=None, subs=[]),
 Sentence(text='MY팀 설정본문 바로가기한겨레데뷔는 지명순이 아니죠…', start=67, end=96, tokens=None, subs=[]),
 Sentence(text='프로농구 루키들, 코트 맹활약 예고입력2025.11.27. 오후 4:14수정2025.11.27. 오후 4:26기사원문공감8텍스트 음성 변환 서비스글자 크기 변경공유하기드래프트 이후 닷새 만에강성욱·이유진 등 프로 데뷔흐름 바꾸는 등 적극성 눈길드래프트 뒤 닷새만에 프로에 데뷔한 강성욱(수원 KT).', start=96, end=263, tokens=None, subs=[]),
 Sentence(text='한국농구연맹 제공“대한민국 가드계에 한 획을 긋겠다”는 야심 찬 계획에 첫 단추를 잘 채웠다.', start=264, end=316, tokens=None, subs=[Sentence(text='대한민국 가드계에 한 획을 긋겠다', start=274, end=292, tokens=None, subs=None)]),
 Sentence(text='지난 14일 신인드래프트 1라운드 8순위로 수원 케이티(KT) 유니폼을 입은 강성욱(21)이 프로 데뷔전에서 눈도장을 제대로 찍었다', start=316, end=389, tokens=None, subs=[]),
 Sentence(text='강성욱은 드래프트 닷새 만인 19일 2025~2026 남자프로농구(KBL) 안양 정관장과 경기에서 8분57초 동안 5득점 2튄공잡기 3도움주기로 팀 승리에 힘을 보탰다.', start=389, end=483, tokens=None, subs=[]),
 Sentence(tex

In [39]:
# 위 분리된 문장에서 띄어쓰기 교정
for sent in result:
    print(kiwi.space(sent.text))

NAVER 스포츠 뉴스 엔터 사용자 링크 내 프로필 이미지 ksryo..
내 정보 보기 네이버 톡 알림 1 메일 99 +메뉴 MY 팀을 설정해 주세요
MY 팀 설정 본문 바로 가기 한 겨레 데뷔는 지명 순이 아니죠…
프로 농구 루키들, 코트 맹활약 예고 입력2025.11.27. 오후 4:14수정2025.11.27. 오후 4:26기사 원문 공감 8 텍스트 음성 변환 서비스 글자 크기 변경 공유하기 드래프트 이후 닷새 만에 강성 욱· 이유진 등 프로 데뷔 흐름 바꾸는 등 적극성 눈길 드래프트 뒤 닷새 만에 프로에 데뷔한 강성 욱(수원 KT).
한국농구연맹 제공“대한민국 가드계에 한 획을 긋겠다”는 야심 찬 계획에 첫 단추를 잘 채웠다.
지난 14일 신인 드래프트 1 라운드 8 순위로 수원 케이티(KT) 유니폼을 입은 강성 욱(21)이 프로 데뷔 전에서 눈도장을 제대로 찍었다
강성 욱은 드래프트 닷새 만인 19일 2025~2026 남자 프로 농구(KBL) 안양 정 관장과 경기에서 8분 57초 동안 5 득점 2 튄 공 잡기 3 도움 주기로 팀 승리에 힘을 보탰다.
후반에 득점과 도움 주기를 연달아 기록하며 흐름을 바꾸는 게임 체인저 역할을 해 냈다
데뷔전에서 수훈 선수에도 뽑혔던 강성 욱은 “나보다 더 높은 순위로 지명된 선수들을 보며 연습에 더 매진하게 됐다”고 말했다.
지난 드래프트에서 지명된 신인들이 하나 둘 프로 무대에 뛰어들면서 올 시즌 농구 보는 재미를 더하고 있다.
지명 순위와 관계 없는 활약에 이번에는 또 어떤 선수가 프로에서 자신의 가치를 증명할지 매 경기 기대감이 커진다.2 라운드 4 순위로 울산 현대모비스에 입단한 박정환(22)도 19일 1 라운드에서 뽑힌 같은 팀 신인보다 먼저 프로 데뷔 전을 치렀다.
서울 에스케이(SK)를 상대로 7분 20초를 뛰며 2 득점 3 도움 주기 1 가로채기했다.
상대 이민 서의 공을 가로챈 뒤 이를 득점으로 연결하는 등 적극적으로 뛰었다
지난 20일 경기에서 프로 데뷔한 원주 디비(DB) 이유진.
한국농구연맹 제공 

In [48]:
txt = "어제 친구가 '집에 가고 싶다.' 라고 이야기 했다."
result = kiwi.split_into_sents(txt) # 결과: list[Sentence]
print(result)
# for sent in result:
#     print(sent)

[Sentence(text="어제 친구가 '집에 가고 싶다.' 라고 이야기 했다.", start=0, end=29, tokens=None, subs=[Sentence(text='집에 가고 싶다.', start=8, end=17, tokens=None, subs=None)])]


In [46]:
print(result[0].text)

어제 친구가 '집에 가고 싶다.' 라고 이야기 했다.


In [47]:
sent = result[0]
if sent.subs != None:
    print(sent.subs[0].text)

집에 가고 싶다.


In [ ]:
##################################################################
# 사전에 사용자단어 추가. - add_user_word(단어, 품사, score)
# - score : 토큰화할 때 그 단어의 우선순위를 조절하는 가중치 값. 클수록 더 선호하게 된다.
#           0: 중립값. 고유명사들은 0을 지정한다.
# - 딥러닝 : 5, 딥: 10, 러닝: 10 -> 딥, 러닝으로 나누는 것을 더 선호하게 된다.
##################################################################
text = "박새로이가 왔다."
kiwi.tokenize(text)

[Token(form='박새로이', tag='NP', start=0, len=4),
 Token(form='가', tag='JKS', start=4, len=1),
 Token(form='오', tag='VV', start=6, len=1),
 Token(form='었', tag='EP', start=6, len=1),
 Token(form='다', tag='EF', start=7, len=1),
 Token(form='.', tag='SF', start=8, len=1)]

In [ ]:
kiwi.add_user_word("박새로이", "NP", 0) # 메모리에 저장해서, 프로그램을 끄면 사라짐
kiwi.tokenize(text)

[Token(form='박새로이', tag='NP', start=0, len=4),
 Token(form='가', tag='JKS', start=4, len=1),
 Token(form='오', tag='VV', start=6, len=1),
 Token(form='었', tag='EP', start=6, len=1),
 Token(form='다', tag='EF', start=7, len=1),
 Token(form='.', tag='SF', start=8, len=1)]

In [56]:
#############################################################
# 불용어(Stop words) 처리 - 토큰화 했을 때 제거할 토큰(단어)들.
#############################################################

# kiwi 제공하는 불용어
from kiwipiepy.utils import Stopwords

sw = Stopwords()
sw

In [58]:
# 불용어 조회
sw.stopwords
len(sw.stopwords)

100

In [59]:
text = "나는 자연어 처리를 공부한다. 자연어처리는 NLP라고 한다."
result = kiwi.tokenize(text)
result

[Token(form='나', tag='NP', start=0, len=1),
 Token(form='는', tag='JX', start=1, len=1),
 Token(form='자연어 처리', tag='NNP', start=3, len=6),
 Token(form='를', tag='JKO', start=9, len=1),
 Token(form='공부', tag='NNG', start=11, len=2),
 Token(form='하', tag='XSV', start=13, len=1),
 Token(form='ᆫ다', tag='EF', start=13, len=2),
 Token(form='.', tag='SF', start=15, len=1),
 Token(form='자연어 처리', tag='NNP', start=17, len=5),
 Token(form='는', tag='JX', start=22, len=1),
 Token(form='NLP', tag='SL', start=24, len=3),
 Token(form='이', tag='VCP', start=27, len=0),
 Token(form='라고', tag='EC', start=27, len=2),
 Token(form='하', tag='VV', start=30, len=1),
 Token(form='ᆫ다', tag='EF', start=30, len=2),
 Token(form='.', tag='SF', start=32, len=1)]

In [60]:
# stopwords 들 제거 - stopwords객체.filter(토큰list)
result2 = sw.filter(result)
result2

[Token(form='자연어 처리', tag='NNP', start=3, len=6),
 Token(form='공부', tag='NNG', start=11, len=2),
 Token(form='자연어 처리', tag='NNP', start=17, len=5),
 Token(form='NLP', tag='SL', start=24, len=3),
 Token(form='라고', tag='EC', start=27, len=2)]

In [61]:
##################
# 불용어 추가/삭제
# sw.add("단어", "품사Tag") #품사 tag 생략 -> NNP(고유명사)
# sw.remove(("단어", "품사Tag")) #
print(kiwi.tokenize("이름이 박새로이입니다."))

[Token(form='이름', tag='NNG', start=0, len=2), Token(form='이', tag='JKS', start=2, len=1), Token(form='박새로이', tag='NNP', start=4, len=4), Token(form='이', tag='VCP', start=8, len=1), Token(form='ᆸ니다', tag='EF', start=8, len=3), Token(form='.', tag='SF', start=11, len=1)]


In [66]:
sw.add(("박새로이", "NNP"))
sw.filter(result)

[Token(form='이름', tag='NNG', start=0, len=2),
 Token(form='ᆸ니다', tag='EF', start=8, len=3)]

In [ ]:
result = kiwi.tokenize("이름이 박새로이입니다.")
print(result)
sw.filter(result)

[Token(form='이름', tag='NNG', start=0, len=2), Token(form='이', tag='JKS', start=2, len=1), Token(form='박새로이', tag='NNP', start=4, len=4), Token(form='이', tag='VCP', start=8, len=1), Token(form='ᆸ니다', tag='EF', start=8, len=3), Token(form='.', tag='SF', start=11, len=1)]


[Token(form='이름', tag='NNG', start=0, len=2),
 Token(form='박새로이', tag='NNP', start=4, len=4),
 Token(form='ᆸ니다', tag='EF', start=8, len=3)]

In [69]:
sw.remove(("박새로이", "NNP"))
sw.filter(result)

ValueError: ('박새로이', 'NNP') doesn't exist in stopwords

In [72]:
# 명사만 추출
result = kiwi.tokenize("나는 NLP 공부를 어제부터 시작했습니다.")
token_list = []
for token in result:
    if token.tag.startswith('N'): #명사는 N으로 시작.
        token_list.append(token)

token_list

[Token(form='나', tag='NP', start=0, len=1),
 Token(form='공부', tag='NNG', start=7, len=2),
 Token(form='어제', tag='NNG', start=11, len=2),
 Token(form='시작', tag='NNG', start=16, len=2)]

## KoNLPy(코엔엘파이)
- KoNLPY는 한국어 자연어 처리(Natural Language Processing) 파이썬 라이브러리이다.  한국어 처리를 위한 tokenize, 형태소 분석, 어간추출, 품사부착(POS Tagging) 등의 기능을 제공한다. 
- http://KoNLPy.org/ko/latest/
- 기존의 개발된 다양한 형태소 분석기를 통합해서 동일한 interface로 호출 할 수 있게 해준다.

### KoNLPy 설치
- 설치 순서
  1. Java 실행환경 설치
  2. JPype1 설치
  3. koNLPy 설치

1. **Java 설치**
  - https://www.oracle.com/java/technologies/downloads/
    - OS에 맞는 설치 버전을 다운받아 설치한다.
    - MAC: ARM일 경우: **ARM64 CPU** - ARM64 DMG Installer, **Intel CPU**: x64 DMG Installer
  - 시스템 환경변수 설정
      - `JAVA_HOME` : 설치 경로 지정
      - `Path` : `설치경로\bin` 경로 지정

2. **JPype1 설치**
   - 파이썬에서 자바 모듈을 호출하기 위한 연동 패키지
   - 설치: `pip install JPype1`

3. **KoNLPy 설치**
- `pip install konlpy`

In [1]:
!uv pip install kiwipiepy

Using Python 3.12.12 environment at: c:\Users\Playdata\Documents\SKN21\.venv
Audited 1 package in 6ms


In [3]:
!uv pip install ipykernel ipywidgets

Using Python 3.12.12 environment at: c:\Users\Playdata\Documents\SKN21\.venv
Resolved 31 packages in 23ms
Installed 3 packages in 48ms
 + ipywidgets==8.1.8
 + jupyterlab-widgets==3.0.16
 + widgetsnbextension==4.0.15


In [73]:
!uv pip install jpype1 konlpy

Using Python 3.12.12 environment at: c:\Users\Playdata\Documents\SKN21\.venv
Resolved 5 packages in 103ms
Prepared 3 packages in 625ms
Installed 3 packages in 46ms
 + jpype1==1.6.0
 + konlpy==0.6.0
 + lxml==6.0.2


In [1]:
import konlpy

: 

### 형태소 분석기/사전
- 형태소 사전을 내장하고 있으며 형태소 분석 함수들을 제공하는 모듈

#### KoNLPy 제공 형태소 분석기
- Open Korean Text
    - 트위터에서 개발
    - https://github.com/open-korean-text/open-korean-text
- Hannanum(한나눔)
    - KAIST Semantic Web Research Center 에서 개발
    - http://semanticweb.kaist.ac.kr/hannanum/
- Kkma(꼬꼬마)
    - 서울대학교 IDS(Intelligent Data Systems) 연구실 개발.
    - http://kkma.snu.ac.kr/
- Komoran(코모란)
    - Shineware에서 개발.
    - 오픈소스버전과 유료버전이 있음
    - https://github.com/shin285/KOMORAN
- Mecab(메카브) 
    - 일본어용 형태소 분석기를 한국에서 사용할 수 있도록 수정
    - windows에서는 설치가 안됨
    - https://bitbucket.org/eunjeon/mecab-ko


### 형태소 분석기 공통 메소드
- `morphs(string)` : 형태소 단위로 토큰화(tokenize)
- `nouns(string)` : 명사만 추출하여 토큰화(tokenize)    
- `pos(string)`: 품사 부착
    - 형태소 분석기 마다 사용하는 품사태그가 다르다.
        - https://konlpy-ko.readthedocs.io/ko/v0.5.2/morph/
- `tagset`: 형태소 분석기가 사용하는 품사태그 설명하는 속성. 

# WordCloud

WordCloud는 텍스트 데이터에서 단어의 등장 빈도를 시각적으로 표현한 그래픽이다.
- 특징
  - 자주 등장하는 단어일수록 글자 크기가 커진다.
  - 텍스트 전체의 주제를 직관적으로 파악할 수 있다.
  - 문서의 핵심키워드를 빠르게 파악할 수 있어 텍스트 분석에서 탐색적 데이터 분석(EDA) 단계에서 자주 활용된다.
- 설치
  - `pip install wordcloud`